In [54]:
import pandas as pd
import numpy as np
import json
import statistics

In [55]:
import os

In [56]:
#Opening up the directory
directory = "/Users/nishantacharya/Downloads/BIRAFFE2/BIRAFFE2-games/BIRAFFE2-games"

In [57]:
test = os.listdir(directory)

In [58]:
File = directory+test[0]

In [59]:
gg = directory+"/"+test[0]

In [60]:
os.listdir(gg)

['SUB200-Level03_Log.json',
 'SUB200-Level02_Log.json',
 'SUB200-Level01_MapLog.json',
 'SUB200-Level02_BlockEvents.json',
 'SUB200-Level01_Log.json']

In [61]:
data = {}

In [62]:
#Collecting the data
for file in os.listdir(directory):
    if 'SUB' in file:
        sub_name = file[3:]
        data[sub_name] = None
        new_d = directory + '/' + file
        for new_f in os.listdir(new_d):
            if "Level01_Log" in new_f:
                jsn = new_d + "/" + new_f
                with open(jsn, 'r') as json_file:
                    json_load = json.load(json_file)
                
                
                # hori,vert,death,shoot,hit,collected money,collected health,sound id, sound stamp, 
                #hori min,vert min,hori maz,vert mazz
                hori_val = []
                vert_val = []
                death_val = []
                shoot_val = []
                hit_val = []
                mny = []
                hlth = []
                sndID = []
                sndTime = []
                hori_min = []
                vert_min = []
                hori_max = []
                vert_max = []
                timestamp = []
                
                
                for i in range(len(json_load)):
                    hori_val.append(json_load[i]['x'])
                    vert_val.append(json_load[i]['y'])
                    timestamp.append(json_load[i]['timestamp'])
                    death_val.append(json_load[i]['deathCount'])
                    shoot_val.append(json_load[i]['shootsCounter'])
                    hit_val.append(json_load[i]['hitCounter'])
                    mny.append(json_load[i]['collectedMoney'])
                    hlth.append(json_load[i]['collectedHealth'])
                    sndID.append(json_load[i]['idOfSound'])
                    sndTime.append(json_load[i]['timestampOfSound'])
                    hori_min.append(json_load[i]['xMin'])
                    vert_min.append(json_load[i]['yMin'])
                    hori_max.append(json_load[i]['xMax'])
                    vert_max.append(json_load[i]['yMax'])
                #Put the data in a dictionary with the SUB title 
                sub_data = {'x':hori_val,'y':vert_val,'timestamp':timestamp,'deathCount':death_val,'shootsCounter':shoot_val,'hitCounter':hit_val,'collectedMoney':mny,'collectedHealth':hlth,'idOfSound':sndID,'timestampOfSound':sndTime,'xMin':hori_min,'yMin':vert_min,'xMax':hori_max,'yMax':vert_max}
                data[sub_name] = sub_data

In [63]:
# function to check if the subject is moving,just ambulating, r is median rows before a change happens
def check_move(smpl,r):
    x_prev = 0
    y_prev = 0
    x_min_prev = 0
    y_min_prev = 0
    x_max_prev = 0
    y_max_prev = 0
    dc_prev = 0
    mc_prev = 0
    hc_prev = 0
    for i in range(smpl.shape[0]):
        #Checking for position change
        if (smpl['x'][i] != x_prev or smpl['y'][i] != y_prev) and smpl['deathCount'][i] == dc_prev and smpl['collectedMoney'][i] == mc_prev and smpl['collectedHealth'][i] == hc_prev:
            n = 0
            x_prev = smpl['x'][i]
            y_prev = smpl['y'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        elif (smpl['xMax'][i] != x_max_prev or smpl['yMax'][i] != y_max_prev) and smpl['deathCount'][i] == dc_prev and smpl['collectedMoney'][i] == mc_prev and smpl['collectedHealth'][i] == hc_prev:
            n = 0
            x_max_prev = smpl['xMax'][i]
            y_max_prev = smpl['yMax'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        elif (smpl['xMin'][i] != x_min_prev or smpl['yMin'][i] != y_min_prev) and smpl['deathCount'][i] == dc_prev and smpl['collectedMoney'][i] == mc_prev and smpl['collectedHealth'][i] == hc_prev:
            n = 0
            x_min_prev = smpl['xMin'][i]
            y_min_prev = smpl['yMin'][i]
            while n < r:#Putting the labels for all values above and below by r rows
                if i + n <smpl.shape[0]:
                    smpl['activityLabel'][i+n] = 1
                if i - n >= 0:
                    smpl['activityLabel'][i-n] = 1
                n+= 1

        else:
            dc_prev = smpl['deathCount'][i]
            mc_prev = smpl['collectedMoney'][i]
            hc_prev = smpl['collectedHealth'][i]

In [64]:
#Function to check for moving and picking up powerups

In [67]:
#Function to check how quickly do the values change
def change_val(frame):
    median_list = []
    for column in frame.columns:
        if 'timestamp' in column:
            continue
        curr_val = float('-inf')
        time = []
        for i in range(frame.shape[0]):
            if curr_val != frame[column][i]:
                curr_val = frame[column][i]
                time.append(frame['timestamp'][i])
        
        difference = np.diff(time)
        median_list.append(statistics.median(difference))
    
    return median_list

In [68]:
import statistics
#Changing the data into Dataframes and adding activities
for key in data:
    smpl = data[key]
    sub_frame = pd.DataFrame(data = smpl)
    
    
    #Finding the average time between the inputs and the number of rows to check before
    md = statistics.median(sub_frame['timestamp'].diff()[1:])
    change = change_val(sub_frame)
    print(md)
    print(change)
    break
    
    
    activity = [-1 for i in range(sub_frame.shape[0])]
    sub_frame["activityLabel"] = activity
    break
    
    data[key] = sub_frame

115.0
[116.0, 116, 32414.5, 217, 217, 2575.0, 9548, 17950, 116, 116.0, 116, 116.0]


In [ ]:
#Double the limit for the item pickup